In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from google.cloud import bigquery
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

In [2]:
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 10000
"""
df = bigquery.Client().query(query).to_dataframe()
df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,6.686620,True,18,1,43.0
1,9.360828,True,32,1,41.0
2,8.437091,False,30,1,39.0
3,6.124442,False,24,1,40.0
4,7.125340,False,26,1,41.0


In [3]:
df = df.dropna()
df = shuffle(df, random_state=2)
labels = df['weight_pounds']
data = df.drop(columns=['weight_pounds'])
data['is_male'] = data['is_male'].astype(int)

In [4]:
x,y = data,labels
x_train,x_test,y_train,y_test = train_test_split(x,y)
model = Sequential([
    Dense(64, activation='relu', input_shape=(len(x_train.iloc[0]),)),
    Dense(32, activation='relu'),
    Dense(1)]
)
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mae', 'mse'])
model.fit(x_train, y_train, epochs=10, validation_split=0.1)

Epoch 1/10
209/209 [==============================] - 1s 4ms/step - loss: 1.8275 - mae: 1.0279 - mse: 1.8275 - val_loss: 1.4841 - val_mae: 0.9644 - val_mse: 1.4841
Epoch 2/10
209/209 [==============================] - 1s 3ms/step - loss: 1.5221 - mae: 0.9648 - mse: 1.5221 - val_loss: 2.8456 - val_mae: 1.4013 - val_mse: 2.8456
Epoch 3/10
209/209 [==============================] - 1s 3ms/step - loss: 1.4691 - mae: 0.9546 - mse: 1.4691 - val_loss: 1.5324 - val_mae: 0.9697 - val_mse: 1.5324
Epoch 4/10
209/209 [==============================] - 1s 3ms/step - loss: 1.3728 - mae: 0.9178 - mse: 1.3728 - val_loss: 1.3518 - val_mae: 0.9132 - val_mse: 1.3518
Epoch 5/10
209/209 [==============================] - 1s 3ms/step - loss: 1.3591 - mae: 0.9137 - mse: 1.3591 - val_loss: 1.8243 - val_mae: 1.0801 - val_mse: 1.8243
Epoch 6/10
209/209 [==============================] - 1s 3ms/step - loss: 1.3569 - mae: 0.9117 - mse: 1.3569 - val_loss: 1.4730 - val_mae: 0.9724 - val_mse: 1.4730
Epoch 7/10
209/2

In [5]:
num_examples = 10
predictions = model.predict(x_test[:num_examples])
for i in range(num_examples):
    print('Predicted val: ', predictions[i][0])
    print('Actual val: ',y_test.iloc[i])
    print()

Predicted val:  7.2062416
Actual val:  5.56226287026

Predicted val:  7.5499907
Actual val:  7.5618555866

Predicted val:  7.4679594
Actual val:  7.5618555866

Predicted val:  7.808281
Actual val:  7.7492485093

Predicted val:  7.4013686
Actual val:  6.0009827716399995

Predicted val:  7.0988555
Actual val:  6.56316153974

Predicted val:  7.64783
Actual val:  6.9996768185

Predicted val:  7.3916264
Actual val:  8.7854211407

Predicted val:  6.988217
Actual val:  6.93794738514

Predicted val:  7.707914
Actual val:  7.68751907594

